See the end of this updated tools and agent notebook for how to use langchain memory as required in HW 4-2

In [20]:
%reload_ext dotenv
%dotenv -o

In [13]:
!pip3 install langchain
!pip3 install langchainhub
!pip3 install langchain_openai

!pip3 list

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is 

In [14]:

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.globals import set_debug
from langchain.agents import AgentExecutor

## Build an agent from scratch

https://python.langchain.com/docs/modules/agents/

In [21]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

Let's create a custom tool and include an explanation so the LLM can understand what it is useful for

In [22]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [23]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

How does the agent know what tools it can use? Those are passed in as a separate argument, so we can bind those as keyword arguments to the LLM.

In [24]:
llm_with_tools = llm.bind_tools(tools)

## Creating memory by hand

In order to do this, we need to do two things:

1. Add a place for memory variables to go in the prompt
2. Keep track of the chat history

First, let's add a place for memory in the prompt. We do this by adding a placeholder for messages with the key `chat_history`. Notice that we put this ABOVE the new user input (to follow the conversation flow).

In [25]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

We can then set up a list to track the chat history

In [26]:
from langchain.schema.messages import HumanMessage, AIMessage
chat_history = []

Putting it together

In [27]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

When running, we now need to track the inputs and outputs as chat history

In [28]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


In [38]:
chat_history

[HumanMessage(content='how many letters in the word educa?'),
 AIMessage(content='There are 5 letters in the word "educa".')]

In [40]:
# input1 = "how many letters in the word educa?"
input1 = "How will I complete this homework assignment?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.append(HumanMessage(content=input1))
chat_history.append(AIMessage(content=result['output']))
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...
To complete your homework assignment, you can follow these steps:

1. Read the assignment instructions carefully to understand what is required.
2. Break down the assignment into smaller tasks or questions.
3. Create a schedule or timeline to manage your time effectively.
4. Gather all the necessary resources, such as textbooks, notes, or online materials.
5. Start working on each task or question one by one, focusing on understanding the concepts and applying them correctly.
6. If you encounter any difficulties or have questions, don't hesitate to ask your teacher or classmates for help.
7. Double-check your work for any errors or mistakes before submitting it.
8. Submit your completed assignment on time, following the submission guidelines provided by your teacher.

Remember to stay organized, manage your time effectively, and seek help when needed. Good luck with your homework assignment!

> Finished chain.


> Entering new AgentExecutor chain

{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".'),
  HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".'),
  HumanMessage(content='How will I complete this homework assignment?'),
  AIMessage(content="To complete your homework assignment, you can follow these steps:\n\n1. Read the assignment instructions carefully to understand what is required.\n2. Break down the assignment into smaller tasks or questions.\n3. Create a schedule or timeline to manage your time effectively.\n4. Gather all the necessary resources, such as textbooks, notes, or online materials.\n5. Start working on each task or question one by one, focusing on understanding the concepts and applying them correctly.\n6. If you encounter any difficulties or have questions, don't hesitate to ask your teacher or classmates for hel

In [42]:
chat_history

[HumanMessage(content='how many letters in the word educa?'),
 AIMessage(content='There are 5 letters in the word "educa".'),
 HumanMessage(content='how many letters in the word educa?'),
 AIMessage(content='There are 5 letters in the word "educa".'),
 HumanMessage(content='How will I complete this homework assignment?'),
 AIMessage(content="To complete your homework assignment, you can follow these steps:\n\n1. Read the assignment instructions carefully to understand what is required.\n2. Break down the assignment into smaller tasks or questions.\n3. Create a schedule or timeline to manage your time effectively.\n4. Gather all the necessary resources, such as textbooks, notes, or online materials.\n5. Start working on each task or question one by one, focusing on understanding the concepts and applying them correctly.\n6. If you encounter any difficulties or have questions, don't hesitate to ask your teacher or classmates for help.\n7. Double-check your work for any errors or mistakes

## Exercise 5-3: Using langchain's memory
**Exercise 5-3 (extra credit):** There are "memory" tools in langchain to manage this easier. Rewrite this example to use it

The above pattern is so common that langchain provides a `ConversationBufferMemory` that does exactly what the above does (updating the `chat_history` key with the previous response) and the `AgentExecutor` constructor accepts a `memory` argument.

The `memory` argument is used [as follows](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html)
> Memory is a class that gets called at the start and at the end of every chain. At the start, memory loads variables and passes them along in the chain. At the end, it saves any returned variables. There are many different types of memory - please see memory docs for the full catalog.

In [34]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor_m = AgentExecutor(agent=agent, tools=tools, memory = memory, verbose=True)

In [36]:
memory.clear() # In case I am rerunning this cell
input1 = "how many letters in the word educe?"
result = agent_executor_m.invoke({"input": input1, "chat_history": chat_history})
agent_executor_m.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educe'}`


5There are 5 letters in the word "educe".

> Finished chain.


> Entering new AgentExecutor chain...
Yes, "educe" is a real word. It means to draw out or bring forth something, especially information or a response.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educe?'),
  AIMessage(content='There are 5 letters in the word "educe".'),
  HumanMessage(content='is that a real word?'),
  AIMessage(content='Yes, "educe" is a real word. It means to draw out or bring forth something, especially information or a response.')],
 'output': 'Yes, "educe" is a real word. It means to draw out or bring forth something, especially information or a response.'}